# <font color='orange'>Librerias</font>

In [ ]:
import pandas as pd
import warnings
import plotly.express as px

warnings.filterwarnings("ignore")

# <font color='orange'>Datos</font>

### <font color='orange'>Coberturas</font>

* Windscreen-> Parabrisas
* DamAll -> Daños por accidente
* Fire -> Fuego/Incendio
* Acc1 -> Accidente de tipo 1
* Acc2 -> Accidente de tipo 2
* Legal -> Defensa jurídica
* TPLM -> Responsabilidad civil obligatoria
* TPLV -> Responsabilidad civil voluntaria
* Serv -> Servicios
* Theft -> Robo

### <font color='orange'>Garantias</font>

| Garantía   | Frecuencia |
|:-----------|:-----------|
| TPL        | 7499       |
| Windscreen | 7092       |
| Damage     | 1951       | 
| Theft      | 1226       |
| Other      | 248        |
| Fire       | 41         |


In [ ]:
# Primas (dataset principal)
df_primas = pd.read_excel('../Data/fremotor1prem.xlsx')
display(df_primas.info(), df_primas.head())

In [ ]:
# Frecuencia
df_freq = pd.read_excel('../Data/fremotor1freq.xlsx')
display(df_freq.info(), df_freq.head())

In [ ]:
# Severidad
df_sev = pd.read_excel('../Data/fremotor1sev.xlsx')
display(df_sev.info(), df_sev.head())

# <font color='orange'>Procesamiento</font>

* Estadísticas
* Gráficas
* Entrenamiento
* Modelos

In [ ]:
# Número de códigos de pólizas únicos
display(len(df_primas['IDpol'].unique()), len(df_freq['IDpol'].unique()), len(df_sev['IDpol'].unique()))

In [ ]:
# Una póliza en particular
df_primas.loc[df_primas['IDpol']=='1000431.10'] #'1000113.100'	

In [ ]:
# Una póliza en particular
df_freq.loc[df_freq['IDpol']=='1000431.10'] #'1000113.100'

In [ ]:
# Una póliza en particular 
df_sev.loc[df_sev['IDpol']=='1000431.10'] #'1000113.100'

In [ ]:
# Comprobación de que la composición de primas es igual a la prima total 
sum(df_primas[['PremWindscreen', 'PremDamAll', 'PremFire', 'PremAcc1',
       'PremAcc2', 'PremLegal', 'PremTPLM', 'PremTPLV', 'PremServ',
       'PremTheft']].sum(axis=1) == df_primas['PremTot'])

In [ ]:
# Chequeo de frecuencias por coberturas
df_sev['Guarantee'].value_counts() 

In [ ]:
# Gráfica de frecuencias de coberturas
frecuencias_cob = df_sev['Guarantee'].value_counts().reset_index()
frecuencias_cob.columns = ['color', 'frecuencia']  

# Crear gráfica de barras
fig = px.bar(frecuencias_cob, x='color', y='frecuencia', title='Frecuencia de coberturas')
fig.show()

#### <font color='orange'>Pólizas con frecuencia máxima 2 y mismo coche</font>

* Motivo: en Francia hay pólizas de diferentes autos con el mismo código (creo que se debe a los casos en los que el tomador es la misma persona).

In [ ]:
# Paso 1: Calcular frecuencia de cada IDpol
frecuencias = df_primas['IDpol'].value_counts()

# Paso 2: Filtrar IDpol con frecuencia máxima de 2
idpol_filtrados = frecuencias[frecuencias <= 2].index

# Paso 3: Filtrar DataFrame con esos IDpol
df_primas_filtrado = df_primas[df_primas['IDpol'].isin(idpol_filtrados)]

# Paso 4: Verificar que cada IDpol tenga un único valor de VehPower
df_primas_final = df_primas_filtrado.groupby('IDpol').filter(lambda g: g['VehPower'].nunique() == 1 and
        set(g['Year']) == {2003, 2004}).reset_index()

display(df_primas_final.info(), df_primas_final.head())

In [ ]:
# Filtramos los datasets de frecuencias y severidad por esos IDpolizas.

polizas_finales = df_primas_final['IDpol'].unique()
df_freq_final = pd.DataFrame(df_freq.loc[df_freq['IDpol'].isin(polizas_finales)])
df_sev_final = pd.DataFrame(df_sev.loc[df_sev['IDpol'].isin(polizas_finales)])

display(df_freq_final.head(), df_sev_final.head())

#### <font color='orange'>Procesamiento de datos</font>

In [ ]:
# Gráfica de frecuencias de coberturas
frecuencias_cob = pd.DataFrame(df_sev_final['Guarantee'].value_counts().reset_index())
frecuencias_cob.columns = ['color', 'frecuencia']  

# Crear gráfica de barras
fig = px.bar(frecuencias_cob, x='color', y='frecuencia', title='Frecuencia de coberturas')
fig.show()
print(frecuencias_cob)

In [ ]:
# Modificación de edad del conductor y edad del vehículo

# Paso 1: Calcular frecuencia de cada IDpol
frecuencias = df_primas_final['IDpol'].value_counts()

# Paso 2: Filtrar IDpol con frecuencia máxima de 2
idpol_filtrados = frecuencias[frecuencias == 2].index

#idpol_filtrados = idpol_filtrados[~idpol_filtrados.isin(['90153350.100', '90152959.100', '90153017.100', '90152947.100'])]


# Paso 3: Ajuste de datos
for idpol in idpol_filtrados:

    #print(idpol)
    if (df_primas_final[(df_primas_final['IDpol']==idpol) & (df_primas_final['Year']==2003)]['VehAge'].reset_index(drop=True) == df_primas_final[(df_primas_final['IDpol']==idpol) & (df_primas_final['Year']==2004)]['VehAge'].reset_index(drop=True)).all():
        df_primas_final.loc[(df_primas_final['IDpol']==idpol) & (df_primas_final['Year']==2004), 'VehAge'] += 1

    if (df_primas_final[(df_primas_final['IDpol']==idpol) & (df_primas_final['Year']==2003)]['DrivAge'].reset_index(drop=True) == df_primas_final[(df_primas_final['IDpol']==idpol) & (df_primas_final['Year']==2004)]['DrivAge'].reset_index(drop=True)).all():
        df_primas_final.loc[(df_primas_final['IDpol']==idpol) & (df_primas_final['Year']==2004), 'DrivAge'] += 1


display(df_primas_final.info(), df_primas_final.head())

In [ ]:
# Control de datasets
display(
    'Número de pólizas únicas ' + str(len(df_primas_final['IDpol'].unique())),
    'Data set de primas ' + str(df_primas_final.shape), 
    'Data set de frecuencias de siniestros ' + str(df_freq_final.shape), 
    'Data set de severidad ' + str(df_sev_final.shape)
    )

In [ ]:
df_freq_final.reset_index(inplace=True)
df_sev_final.reset_index(inplace=True)

In [ ]:
# Unión de los data sets de primas y frecuencias

# Paso 1: combinar ambos DataFrames por IDpol
df = df_primas_final.merge(df_freq_final, on='IDpol', how='left')

# Paso 2: crear la nueva columna 'ClaimNb' según el año
df['ClaimNb'] = df.apply(
    lambda row: row['ClaimNb2003'] if row['Year'] == 2003 else row['ClaimNb2004'],
    axis=1
)

# (Opcional) Paso 3: eliminar las columnas intermedias si ya no se necesitan
df = df.drop(columns=['ClaimNb2003', 'ClaimNb2004'])

df.head()

In [ ]:
# Vamos a introducir los tipos de siniestros que haya tenido cada póliza en la tabla primas

# Creamos la variable year en la tabla de severidad
df_sev_final['year'] = df_sev_final['OccurDate'].dt.year

# Creamos las columnas por siniestros / coberturas (garantias) y costes de siniestros / coberturas (garantias) en la tabla de primas
    # Guarantee
    # ['Windscreen', 'Theft', 'Damage', 'TPL', 'Other', 'Fire']
df['Windscreen_Claims'] = 0
df['Theft_Claims'] = 0
df['Damage_Claims'] = 0
df['TPL_Claims'] = 0
df['Other_Claims'] = 0
df['Fire_Claims'] = 0

df['Windscreen_Claims_Amount'] = 0
df['Theft_Claims_Amount'] = 0
df['Damage_Claims_Amount'] = 0
df['TPL_Claims_Amount'] = 0
df['Other_Claims_Amount'] = 0
df['Fire_Claims_Amount'] = 0

df['Windscreen_Claims_Sev'] = 0
df['Theft_Claims_Sev'] = 0
df['Damage_Claims_Sev'] = 0
df['TPL_Claims_Sev'] = 0
df['Other_Claims_Sev'] = 0
df['Fire_Claims_Sev'] = 0

# Recorremos la tabla de severidad para autocompletar la de primas (más optimizado)
for index, row in df_sev_final.iterrows():

    #print(f"Póliza: {row['IDpol']}, Año: {row['year']}, Cobertura: {row['Guarantee']}, Coste: {row['Payment']}")
    
    freq_col = row['Guarantee']+'_Claims'
    sev_col = row['Guarantee']+'_Claims_Amount'
    
    df.loc[(df['IDpol']==row['IDpol']) & (df['Year']==row['year']), freq_col] += 1
    df.loc[(df['IDpol']==row['IDpol']) & (df['Year']==row['year']), sev_col] += row['Payment']


for index, row in df.iterrows():

    if row['Windscreen_Claims']!=0:

        df.loc[index, 'Windscreen_Claims_Sev'] = row['Windscreen_Claims_Amount']/row['Windscreen_Claims'] 
    
    if row['Theft_Claims']!=0:

        df.loc[index, 'Theft_Claims_Sev'] = row['Theft_Claims_Amount']/row['Theft_Claims']

    if row['Damage_Claims']!=0:

        df.loc[index, 'Damage_Claims_Sev'] = row['Damage_Claims_Amount']/row['Damage_Claims']

    if row['TPL_Claims']!=0:

        df.loc[index, 'TPL_Claims_Sev'] = row['TPL_Claims_Amount']/row['TPL_Claims']

    if row['Other_Claims']!=0:

        df.loc[index, 'Other_Claims_Sev'] = row['Other_Claims_Amount']/row['Other_Claims']

    if row['Fire_Claims']!=0:

        df.loc[index, 'Fire_Claims_Sev'] = row['Fire_Claims_Amount']/row['Fire_Claims']



display(df.info(), df.head())

#### <font color='orange'>Variables explicativas</font>

* Crear una variable ID con valores únicos (propuesta: IDpol + Year).

'IDpol', 'Year', 
'DrivAge', 'DrivGender', 'MaritalStatus', 'BonusMalus', 'LicenceNb', 'VehNb', 'PayFreq', 'JobCode', 'VehAge',
'VehClass', 'VehPower', 'VehGas', 'VehUsage', 'Garage', 'Area',  'Region', 'Channel', 'Marketing', 'PremWindscreen', 'PremDamAll',
'PremFire', 'PremAcc1', 'PremAcc2', 'PremLegal', 'PremTPLM', 'PremTPLV', 'PremServ', 'PremTheft', 'PremTot'

#### <font color='orange'>Variables objetivo</font>

'Windscreen_Claims', 'Theft_Claims', 'Damage_Claims', 'TPL_Claims', 'Other_Claims', 'Fire_Claims',
'Windscreen_Claims_Sev', 'Theft_Claims_Sev', 'Damage_Claims_Sev', 'TPL_Claims_Sev', 'Other_Claims_Sev', 'Fire_Claims_Sev'

In [ ]:
print(df[['Windscreen_Claims', 'Theft_Claims', 'Damage_Claims', 'TPL_Claims', 'Other_Claims', 'Fire_Claims']].sum().sort_values(ascending=False))
print(df[['Windscreen_Claims_Sev', 'Theft_Claims_Sev', 'Damage_Claims_Sev', 'TPL_Claims_Sev', 'Other_Claims_Sev', 'Fire_Claims_Sev']].sum().sort_values(ascending=False))

#### <font color='orange'>Creamos los data set para cada uno de los problemas (las 4 coberturas más frecuentes)</font>

In [ ]:
df['IDpol_Year'] = df['IDpol'] + '_' + df['Year'].astype(str)
features = ['IDpol_Year', 'DrivAge', 'DrivGender', 'MaritalStatus', 'BonusMalus', 'LicenceNb', 'VehNb', 'PayFreq', 'JobCode', 'VehAge', 
            'VehClass', 'VehPower', 'VehGas', 'VehUsage', 'Garage', 'Area',  'Region', 'Channel', 'Marketing', 'PremWindscreen', 'PremDamAll',
            'PremFire', 'PremAcc1', 'PremAcc2', 'PremLegal', 'PremTPLM', 'PremTPLV', 'PremServ', 'PremTheft', 'PremTot']

# Data sets de frecuencias
df[features + ['Windscreen_Claims']].to_excel('df_windscreen_freq.xlsx', index=False)
df[features + ['TPL_Claims']].to_excel('df_tpl_freq.xlsx', index=False)
df[features + ['Damage_Claims']].to_excel('df_damage_freq.xlsx', index=False)
df[features + ['Theft_Claims']].to_excel('df_theft_freq.xlsx', index=False)

# Data sets de severidad
df[features + ['Windscreen_Claims_Sev']].to_excel('df_windscreen_sev.xlsx', index=False)
df[features + ['TPL_Claims_Sev']].to_excel('df_tpl_sev.xlsx', index=False)
df[features + ['Damage_Claims_Sev']].to_excel('df_damage_sev.xlsx', index=False)
df[features + ['Theft_Claims_Sev']].to_excel('df_theft_sev.xlsx', index=False)
